In [1]:
import pandas as pd
import numpy as np
from flask import Flask
import pickle 
from flask import request,redirect

In [2]:
def pasingdata(data_dict):
    car_data = []
    car_data.append(data_dict["brand"])
    car_data.append(data_dict["model"])
    car_data.append(data_dict["d_model"])
    car_data.append(data_dict["grade"])
    car_data.append(data_dict["year"])
    return car_data

In [3]:
## 받은 데이터를 학습시킬수있게 변환해주는 함수
car_col = ['신차가격', '연식', '주행거리', '변속기', '연비', '배기량', '압류', '저당', '전손이력', '용도이력',
       '사용자변경', '차체/일반', '엔진/주요', '최고출력', '최대토크', '승차정원', '휠_1', '타이어_수정',
       '연료_LPG', '연료_가솔린', '연료_기타', '연료_디젤', '차종_RV', '차종_SUV', '차종_경차',
       '차종_대형', '차종_소형', '차종_승합', '차종_준중형', '차종_중형', '차종_트럭', '색상_검정색',
       '색상_기타', '색상_은색', '색상_진주색', '색상_회색', '색상_흰색', '제조사_기아', '제조사_르노삼성',
       '제조사_쌍용', '제조사_제네시스', '제조사_한국GM', '제조사_현대', '구동방식_4WD', '구동방식_AWD',
       '구동방식_FF', '구동방식_FR']

def changedata(data_dict):
    ## 학습데이터 컬럼수 47 
    # 2021 1만km
    # 20 2만키로
    # 19 4만키로
    # 21 - df["연식"] => 1 * 20000
    # 연식 16  주행거리 /1000 => 1 2 3 4 5 6
    # 차체일반은 3/6 엔진은 5/10 넘으면 만료니까 0으로 줌
    # 색깔  흰색', '기타', '진주색', '검정색', '회색', '은색'
    # ======
    # 연료 채워야됨
    # 변속기 기본 오토
    # 차종은 일단 더미로하고
    
    df = pd.DataFrame(np.zeros((1,47)), columns=car_col)
    df["신차가격"] = 2228
    df["연식"] = int(data_dict['year'])%100
    df["주행거리"] = (21 - int(data_dict['year'])%100)*20000
    
    if data_dict['gear']=="오토":
        df["변속기"] = 1
    else :
        df["변속기"] = 0
        
    df["연비"] = data_dict["fe"]
    df["배기량"] = int(data_dict["cc"])
    df["최고출력"] = int(data_dict["output"])
    df["최대토크"] = int(data_dict["torque"])
    df["승차정원"] = int(data_dict["people"])
    df["휠_1"] = int(data_dict["wheel"])
    df["타이어_수정"] = int(data_dict["tire"])
    
    df["압류"] = 0
    df["저당"] = 0
    df["전손이력"] = 0
    df["용도이력"] = 0
    df["사용자변경"] = 0
    df["차체/일반"] = 0
    df["엔진/주요"] = 1
    
    
    ## 값으로 들어온것만 1 나머지 0
    df["연료_"+data_dict["fuel"]] = 1
    if data_dict["car_type"] =="준대형":
        df["차종_대형"] = 1
    else:
        df["차종_"+data_dict["car_type"]] = 1
    
    df["색상_"+data_dict["color"]] = 1
    df["제조사_"+data_dict["brand"]] = 1
    df["구동방식_"+data_dict["drivesys"]] = 1
    print(df.columns)
    return df
    
    

In [6]:
app = Flask(__name__) # 내장변수 name을 이용해 서버 구동시키는 객체 생성

with open("xgb_model_complete.model","rb") as f:
    model = pickle.load(f)
df1=0
@app.route("/carpred", methods=["POST","GET"])## 값 받을 파이썬 주소
def predict_price():
    if request.method == "POST":
        display(request.form)
        
        # 넘어온 값 전처리()
        d_list = pasingdata(request.form)
        if d_list[0] == "" or d_list[1] =="" or d_list[2] == "" or d_list[3] == "" or d_list[4] == "":
            return redirect("http://localhost:8082/CarSite/carprice.jsp")
        df = changedata(request.form)
        df1=df
        print(df)
        pre_list = []
        for i in range(5) :
            df["연식"] = df["연식"] -i
            df["주행거리"] = df["주행거리"] + (i*20000)
            if df.iloc[0,1]<=18 :
                df["차체/일반"] = 0
            else :
                df["차체/일반"] = 1
            if df.iloc[0,1]<=16 :
                df["엔진/주요"] = 0
            else :
                df["엔진/주요"] = 1
            pre = model.predict(df)
            preStr = str(pre[0]).split(".")[0]
            pre_list.append(preStr)
                
        print(f"http://localhost:8082/CarSite/pricePre?brand={d_list[0]}&model={d_list[1]}&d_model={d_list[2]}&grade={d_list[3]}&year={d_list[4]}&pre={preStr}")
        return redirect(f"http://localhost:8082/CarSite/pricePre?brand={d_list[0]}&model={d_list[1]}&d_model={d_list[2]}&grade={d_list[3]}&year={d_list[4]}&pre={pre_list[0]}&pre2={pre_list[1]}&pre3={pre_list[2]}&pre4={pre_list[3]}&pre5={pre_list[4]}")

if __name__ == "__main__" : # .py 파일에서 main 함수 역할
    app.run(host="localhost",port="9000") ## app.run("서버 url,포트번호")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


ImmutableMultiDict([('brand', '기아'), ('model', '뉴모닝'), ('d_model', 'LX'), ('grade', '기본형'), ('year', '2008'), ('color', '흰색'), ('car_type', '경차'), ('gear', '오토'), ('fuel', '가솔린'), ('fe', '19'), ('cc', '999'), ('output', '64'), ('torque', '9'), ('drivesys', 'FF'), ('people', '5'), ('wheel', '13'), ('tire', '175')])

127.0.0.1 - - [28/Jun/2021 09:41:01] "POST /carpred HTTP/1.1" 302 -


Index(['신차가격', '연식', '주행거리', '변속기', '연비', '배기량', '압류', '저당', '전손이력', '용도이력',
       '사용자변경', '차체/일반', '엔진/주요', '최고출력', '최대토크', '승차정원', '휠_1', '타이어_수정',
       '연료_LPG', '연료_가솔린', '연료_기타', '연료_디젤', '차종_RV', '차종_SUV', '차종_경차',
       '차종_대형', '차종_소형', '차종_승합', '차종_준중형', '차종_중형', '차종_트럭', '색상_검정색',
       '색상_기타', '색상_은색', '색상_진주색', '색상_회색', '색상_흰색', '제조사_기아', '제조사_르노삼성',
       '제조사_쌍용', '제조사_제네시스', '제조사_한국GM', '제조사_현대', '구동방식_4WD', '구동방식_AWD',
       '구동방식_FF', '구동방식_FR'],
      dtype='object')
   신차가격  연식    주행거리  변속기  연비  배기량  압류  저당  전손이력  용도이력  ...  제조사_기아  제조사_르노삼성  \
0  2228   8  260000    1  19  999   0   0     0     0  ...       1       0.0   

   제조사_쌍용  제조사_제네시스  제조사_한국GM  제조사_현대  구동방식_4WD  구동방식_AWD  구동방식_FF  구동방식_FR  
0     0.0       0.0       0.0     0.0       0.0       0.0        1      0.0  

[1 rows x 47 columns]
http://localhost:8082/CarSite/pricePre?brand=기아&model=뉴모닝&d_model=LX&grade=기본형&year=2008&pre=331
